In [ ]:
from qat.core import Observable as Obs, Term
from qat.lang import AbstractGate, qrout, Program, X, Y, Z, H, S, RX, RY, RZ, CNOT, CSIGN, PH, SWAP, QRoutine
from qat.qpus import LinAlg, NoisyQProc
from qat.hardware import HardwareModel, GatesSpecification, DefaultGatesSpecification, DefaultHardwareModel
from qat.quops import ParametricAmplitudeDamping, ParametricGateNoise

import numpy as np
import toolbox as tb
from scipy import integrate
from itertools import product
from matplotlib import pyplot as plt
from datetime import datetime

from quantum_algo.correlation_circuit import circuits_for_correlator, evaluate_circuits
from quantum_algo.lindblad_noisy_algo import trotter_lindblad_noise, make_hardware_model, noise_as_a_resource_simulation
from quantum_algo.resonnant_model import make_resonnant_model_ham, make_resonnant_model_hpq
from bath_fit.closed_bath import closed_bath_fit
from bath_fit.open_bath import fit_open_bath_physm, plot_fit

from toolbox.free_fermions import SlaterDetState, free_fermions_observable, free_fermions_greater, free_fermions_observable_lindblad, free_fermions_greater_lindblad
from toolbox.free_fermions import find_gibbs_state_rdm_free_fermions


In [ ]:
### Parameters

coupling1 = 0.2
coupling2 = 0.6
eps = 0.5
beta = 1.0
half_bandwidth = 10.


In [ ]:
### target hybridization functions

def target_dos_unnorm(omega):
    if abs(omega) < half_bandwidth:
        return 1. / (2 * half_bandwidth)
    else:
        return 0.0

def target_dos1(omega):
    return coupling1**2 * target_dos_unnorm(omega)
    
def target_dos2(omega):
    return coupling2**2 * target_dos_unnorm(omega)
    
target_dos1 = np.vectorize(target_dos1)
target_dos2 = np.vectorize(target_dos2)
print(integrate.quad(target_dos1, -20., 20.), "==", coupling1**2)

def target_less1(omega):
    return 2 * np.pi * tb.fermi(omega, 0., beta) * target_dos1(omega)

def target_grea1(omega):
    return 2 * np.pi * tb.fermi(omega, 0., -beta) * target_dos1(omega)

def target_less2(omega):
    return 2 * np.pi * tb.fermi(omega, 0., beta) * target_dos2(omega)

def target_grea2(omega):
    return 2 * np.pi * tb.fermi(omega, 0., -beta) * target_dos2(omega)

omegas = np.linspace(-15, 15, 300)

plt.plot(omegas, target_dos1(omegas))
plt.show()
plt.plot(omegas, target_less1(omegas))
plt.plot(omegas, target_grea1(omegas))
plt.show()


In [ ]:
start = datetime.now()

time_list_noiseless = np.linspace(0, 500, 1000)
nb_bath_list_noiseless = [50, 100, 150, 200]#, 2000]
gf_grea1_arr_noiseless = np.empty((len(nb_bath_list_noiseless), len(time_list_noiseless)), dtype=complex)

def density(omega):
    return np.exp(-omega**2 / (2 * 5**2))

for k, nb_bath in enumerate(nb_bath_list_noiseless):
    eps_bath, v_bath = closed_bath_fit(target_dos1, nb_bath, density, break_points=[-half_bandwidth, half_bandwidth])

    hpq_unitary = make_resonnant_model_hpq(eps, v_bath, eps_bath)
    # rdm_gs = find_ground_state_rdm_free_fermions(hpq_unitary)
    rdm_gs = find_gibbs_state_rdm_free_fermions(hpq_unitary, beta=beta)
    print("nb of fermions:", np.trace(rdm_gs))

    gf_grea1_arr_noiseless[k] = free_fermions_greater(hpq_unitary, 0, 0, rdm_gs, time_list_noiseless, 0.)
    
print("runtime:", datetime.now() - start)

In [ ]:
start = datetime.now()

gf_grea2_arr_noiseless = np.empty((len(nb_bath_list_noiseless), len(time_list_noiseless)), dtype=complex)

for k, nb_bath in enumerate(nb_bath_list_noiseless):
    eps_bath, v_bath = closed_bath_fit(target_dos2, nb_bath, density, break_points=[-half_bandwidth, half_bandwidth])

    hpq_unitary = make_resonnant_model_hpq(eps, v_bath, eps_bath)
    # rdm_gs = find_ground_state_rdm_free_fermions(hpq_unitary)
    rdm_gs = find_gibbs_state_rdm_free_fermions(hpq_unitary, beta=beta)
    print("nb of fermions:", np.trace(rdm_gs))

    gf_grea2_arr_noiseless[k] = free_fermions_greater(hpq_unitary, 0, 0, rdm_gs, time_list_noiseless, 0.)
    
print("runtime:", datetime.now() - start)

In [ ]:

for k, nb_bath in enumerate(nb_bath_list_noiseless):
    plt.plot(time_list_noiseless, gf_grea2_arr_noiseless[k].real, label=f"nb bath = {nb_bath}")
    
plt.legend()
# plt.loglog()
plt.xlim(0, 200)
plt.xlabel("t")
plt.ylabel("G^>(t)")
# plt.savefig("quantum_algo/revivals.png", dpi=250)
plt.show()

## open bath

In [ ]:
nb_bath = 20


fit = fit_open_bath_physm(target_grea2, 
                          nb_bath, 
                          spectral_tol=0.1 * coupling2**2,
                          omega_max=half_bandwidth,
                          break_points=[-half_bandwidth, half_bandwidth],
                       )

eps_absorb, eps_emitt, v_absorb, v_emitt, dissip_rate, chi_sqr = fit

print("unused bath sites:", sum(np.abs(v_absorb) < 1e-10))

plt.axvline(eps, c='k', ls=':', alpha=0.3)
plot_fit(target_dos2, np.append(eps_absorb, eps_emitt), np.append(v_absorb, v_emitt)**2 * dissip_rate / np.pi, dissip_rate)

plt.axvline(eps, c='k', ls=':', alpha=0.3)
plot_fit(target_less2, eps_emitt, 2 * v_emitt**2 * dissip_rate, dissip_rate)

print("dissip rate =", dissip_rate)
print("chi^2 =", chi_sqr)


In [ ]:
nb_bath_list_noisy = [16, 20, 24, 28, 32]
tprime1_list = np.linspace(0, 500, 6)
time_list_noisy = np.linspace(0, 500, 1000)
gf_grea1_arr_noisy = np.empty((len(nb_bath_list_noisy), len(tprime1_list), len(time_list_noisy)), dtype=complex)

for k, nb_bath in enumerate(nb_bath_list_noisy):
    fit = fit_open_bath_physm(target_grea1, 
                              nb_bath, 
                              spectral_tol=0.1 * coupling1**2,
                              omega_max=half_bandwidth,
                              break_points=[-half_bandwidth, half_bandwidth],
                             )

    eps_absorb, eps_emitt, v_absorb, v_emitt, dissip_rate, chi_sqr = fit
    print("unused bath sites:", sum(np.abs(v_absorb) < 1e-10))
    print("dissip rate =", dissip_rate)
    print("chi^2 =", chi_sqr)

    hpq_open = make_resonnant_model_hpq(eps, np.append(v_emitt, v_absorb), np.append(eps_emitt, eps_absorb))
    rdm_init = SlaterDetState("0" + len(eps_emitt) * "1" + len(eps_absorb) * "0").get_rdm()
    lam_minus = dissip_rate * np.diag([0.] + len(eps_emitt) * [0.] + len(eps_absorb) * [1.])
    lam_plus  = dissip_rate * np.diag([0.] + len(eps_emitt) * [1.] + len(eps_absorb) * [0.])

    for l, tprime in enumerate(tprime1_list):
        gf_grea1_arr_noisy[k, l, :] = free_fermions_greater_lindblad(hpq_open, 
                                                              lam_minus, 
                                                              lam_plus, 
                                                              rdm_init, 
                                                              time_list_noisy + tprime, 
                                                              tprime)[:, 0, 0]
    
    plt.plot(time_list_noisy, gf_grea1_arr_noisy[k, -1].real, label=f"nb bath = {nb_bath}, t' = {tprime}")
    
plt.legend()
plt.show()


In [ ]:

tprime2_list = tprime1_list / 5.
gf_grea2_arr_noisy = np.empty((len(nb_bath_list_noisy), len(tprime2_list), len(time_list_noisy)), dtype=complex)

for k, nb_bath in enumerate(nb_bath_list_noisy):
    fit = fit_open_bath_physm(target_grea2, 
                              nb_bath, 
                              spectral_tol=0.1 * coupling2**2,
                              omega_max=half_bandwidth,
                              break_points=[-half_bandwidth, half_bandwidth],
                             )

    eps_absorb, eps_emitt, v_absorb, v_emitt, dissip_rate, chi_sqr = fit
    print("unused bath sites:", sum(np.abs(v_absorb) < 1e-10))
    print("dissip rate =", dissip_rate)
    print("chi^2 =", chi_sqr)

    hpq_open = make_resonnant_model_hpq(eps, np.append(v_emitt, v_absorb), np.append(eps_emitt, eps_absorb))
    rdm_init = SlaterDetState("0" + len(eps_emitt) * "1" + len(eps_absorb) * "0").get_rdm()
    lam_minus = dissip_rate * np.diag([0.] + len(eps_emitt) * [0.] + len(eps_absorb) * [1.])
    lam_plus  = dissip_rate * np.diag([0.] + len(eps_emitt) * [1.] + len(eps_absorb) * [0.])

    for l, tprime in enumerate(tprime2_list):
        gf_grea2_arr_noisy[k, l, :] = free_fermions_greater_lindblad(hpq_open, 
                                                              lam_minus, 
                                                              lam_plus, 
                                                              rdm_init, 
                                                              time_list_noisy + tprime, 
                                                              tprime)[:, 0, 0]
    
    plt.plot(time_list_noisy, gf_grea2_arr_noisy[k, -1].real, label=f"nb bath = {nb_bath}, t' = {tprime}")
    
plt.legend()
plt.show()


In [ ]:

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(4, 4))
fig.suptitle("Absolute error on $G^>(t-t')$")

for i, idx in enumerate([10, 30]):
    plt.sca(ax[i])
    plt.semilogy()
    plt.xlabel(r"$t' \; \Delta(\varepsilon)$")
    tb.plot.autotext(rf"$t-t' = {tb.significant_round(time_list_noisy[idx], 2)}$")

    for k in [0, -1]:
        nb_bath = nb_bath_list_noisy[k]
        plt.plot(tprime2_list[:-1] * target_dos2(eps), np.abs(gf_grea2_arr_noisy[k, :, idx] - gf_grea2_arr_noisy[k, -1, idx])[:-1], 
                 'o-', label=f"$\gamma = {coupling2}$, $N_b = {nb_bath}$")

        plt.plot(tprime1_list[:-1] * target_dos1(eps), np.abs(gf_grea1_arr_noisy[k, :, idx] - gf_grea1_arr_noisy[k, -1, idx])[:-1], 
                 '^--', label=f"$\gamma = {coupling1}$, $N_b = {nb_bath}$")

plt.sca(ax[0])
plt.legend(loc="lower left", fontsize=8)
plt.tight_layout()
# plt.savefig("quantum_algo/figure_convergence_tprime.png", dpi=300)
# plt.savefig("quantum_algo/figure_convergence_tprime.pdf")
plt.show()

## Comparison noisy vs noiseless

In [ ]:
### comp noisy and noiseless
plt.figure(0, (15, 5))

plt.plot(time_list_noisy, gf_grea1_arr_noisy[-1, -1].real, label=f"noisy, nb bath = {nb_bath_list_noisy[-1]}, t' = {tprime1_list[-1]}")
plt.plot(time_list_noiseless, gf_grea1_arr_noiseless[-2].real, label=f"noiseless, nb bath = {nb_bath_list_noiseless[-2]}")
plt.plot(time_list_noiseless, gf_grea1_arr_noiseless[-1].real, label=f"noiseless, nb bath = {nb_bath_list_noiseless[-1]}")
    
plt.legend()
plt.show()

In [ ]:
### comp noisy and noiseless
plt.figure(0, (15, 5))

plt.plot(time_list_noisy, gf_grea2_arr_noisy[-1, -1].real, label=f"noisy, nb bath = {nb_bath_list_noisy[-1]}, t' = {tprime2_list[-1]}")

for k, nb_bath in enumerate(nb_bath_list_noiseless):
    plt.plot(time_list_noiseless, gf_grea2_arr_noiseless[k].real, label=f"noiseless, nb bath = {nb_bath}")
# plt.plot(time_list_noiseless, gf_grea2_arr_noiseless[-1].real, label=f"noiseless, nb bath = {nb_bath_list_noiseless[-1]}")
    
plt.xlim(0, 120)
plt.legend()
plt.show()

In [ ]:
### comp noisy and noiseless
omega1_noisy, gf_grea1_noisy = tb.fourier_transform(*tb.symmetrize(time_list_noisy, 
                                                                 gf_grea1_arr_noisy, 
                                                                 0., lambda x: -np.conj(x), axis=-1),
                                                  n=None,
                                                  axis=-1)
omega1_noiseless, gf_grea1_noiseless = tb.fourier_transform(*tb.symmetrize(time_list_noiseless, 
                                                                         gf_grea1_arr_noiseless, 
                                                                         0., lambda x: -np.conj(x), axis=-1),
                                                          n=None,
                                                          axis=-1)

omega1_noiseless_short, gf_grea1_noiseless_short = tb.fourier_transform(*tb.vcut(*tb.symmetrize(time_list_noiseless, 
                                                                                    gf_grea1_arr_noiseless, 
                                                                                    0., lambda x: -np.conj(x), axis=-1),
                                                                     -100., 100., axis=-1),
                                                          n=None,
                                                          axis=-1)

omega2_noisy, gf_grea2_noisy = tb.fourier_transform(*tb.symmetrize(time_list_noisy, 
                                                                 gf_grea2_arr_noisy, 
                                                                 0., lambda x: -np.conj(x), axis=-1),
                                                  n=None,
                                                  axis=-1)
omega2_noiseless, gf_grea2_noiseless = tb.fourier_transform(*tb.symmetrize(time_list_noiseless, 
                                                                         gf_grea2_arr_noiseless, 
                                                                         0., lambda x: -np.conj(x), axis=-1),
                                                          n=None,
                                                          axis=-1)

omega2_noiseless_short, gf_grea2_noiseless_short = tb.fourier_transform(*tb.vcut(*tb.symmetrize(time_list_noiseless, 
                                                                                    gf_grea2_arr_noiseless, 
                                                                                    0., lambda x: -np.conj(x), axis=-1),
                                                                     -25., 25., axis=-1),
                                                          n=None,
                                                          axis=-1)



In [ ]:


plt.plot(omega1_noiseless, -gf_grea1_noiseless[-1].imag, "--k", label=f"noiseless, nb bath = {nb_bath_list_noiseless[-1]}")
    
plt.plot(omega1_noiseless_short, -gf_grea1_noiseless_short[-2].imag, label=f"noiseless, nb bath = {nb_bath_list_noiseless[-2]}")
    
plt.plot(omega1_noisy, -gf_grea1_noisy[-4, -1].imag, label=f"noisy, nb bath = {nb_bath_list_noisy[-4]}")
    
plt.plot(omega1_noisy, -gf_grea1_noisy[-1, -1].imag, label=f"noisy, nb bath = {nb_bath_list_noisy[-1]}")
    
plt.xlim(0.35, 0.75)
plt.legend()
plt.xlabel(r"$\omega$")
plt.ylabel(r"$G^>(\omega)$")
plt.show()

In [ ]:


# plt.plot(omega1_noiseless, -gf_grea1_noiseless[-1].imag, "--k", label=f"noiseless, nb bath = {nb_bath_list_noiseless[-1]}")
    
plt.plot(omega1_noiseless_short, -gf_grea1_noiseless_short[0].imag, label=f"noiseless, nb bath = {nb_bath_list_noiseless[0]}")
    
for k, nb_bath in enumerate(nb_bath_list_noisy):
    # if k % 2 == 1:
    #     continue
    plt.plot(omega1_noisy, -gf_grea1_noisy[k, -1].imag, label=f"noisy, nb bath = {nb_bath}")
        
plt.xlim(0.4, 0.65)
plt.legend()
plt.xlabel(r"$\omega$")
plt.ylabel(r"$G^>(\omega)$")
plt.show()

In [ ]:


# plt.plot(omega2_noiseless, -gf_grea2_noiseless[-1].imag, "--k", label=f"noiseless, nb bath = {nb_bath_list_noiseless[-1]}")
    
plt.plot(omega2_noiseless_short, -gf_grea2_noiseless_short[0].imag, label=f"noiseless, nb bath = {nb_bath_list_noiseless[0]}")
    
for k, nb_bath in enumerate(nb_bath_list_noisy):
    plt.plot(omega2_noisy, -gf_grea2_noisy[k, -1].imag, label=f"noisy, nb bath = {nb_bath}")
        
plt.xlim(-0., 1.5)
plt.legend()
plt.xlabel(r"$\omega$")
plt.ylabel(r"$G^>(\omega)$")
plt.show()

In [ ]:
### FIGURE

plt.figure(0, (5, 5))
plt.subplot(2, 1, 1)

for k, nb_bath in enumerate(nb_bath_list_noiseless):
    if nb_bath > 170:
        continue
    plt.plot(time_list_noiseless, gf_grea2_arr_noiseless[k].real, label=rf"$N_b = {nb_bath}$")
    
plt.plot(time_list_noisy, gf_grea2_arr_noisy[-1, -1].real, "--k",
         label=rf"$N_b = {nb_bath_list_noisy[-1]}$ (noisy)")

plt.xlim(-1, 90)
plt.ylim(-0.55, 0.8)
plt.xlabel(r"$t - t'$")
plt.ylabel(r"$\Re[G^>(t - t')]$")
plt.legend(loc="upper left", 
           ncols=2, 
           numpoints=2, 
           columnspacing=0.5, 
           labelspacing=0.1,
           frameon=False,
          )


plt.subplot(2, 1, 2)
c = tb.color_list(nb_bath_list_noisy, cmap_name="copper_r")
    
for k, nb_bath in enumerate(nb_bath_list_noisy):
    plt.plot(omega2_noisy, -gf_grea2_noisy[k, -1].imag,
             c=c[k], label=rf"$N_b = {nb_bath}$")
        
plt.plot(omega2_noiseless_short, -gf_grea2_noiseless_short[0].imag, '.-',
         label=rf"$N_b = {nb_bath_list_noiseless[0]}$ (noiseless)")
    
plt.xlim(-0., 1.5)
plt.legend(loc="upper right", 
           ncols=1, 
           numpoints=2, 
           columnspacing=0.5, 
           labelspacing=0.1,
           frameon=False,
          )
plt.xlabel(r"$\omega$")
plt.ylabel(r"$-\Im[G^>(\omega)]$")

plt.tight_layout()
# plt.savefig("quantum_algo/figure_noisy_vs_noiseless_analytic.pdf")
plt.show()